## 発表内容

最近日本語対応したAmazon KendraはAmazon S3やAmazon RDS、Salesforceといったリポジトリのコンテンツを一元管理されたインデックスに簡単に集約できます。これにより企業の様々なデータを素早く検索できます。そして、Amazon Comprehendはドキュメント内のテキストの解析を行うことができます。この2つのサービスを組み合わせた文書検索と検索された文書の解析についてデモを交えてご紹介します。

----

## 概要

- 非構造化データは増え続けている
- 非構造データから必要な情報を入手することが困難
- Kendraを使うと非構造データの検索自体は可能
- Kendraでジャンル等の絞り込みフィルタリングを行うことでより精度の高い検索実現したい・・・・

### 解決策（一例）

- Kendraではドキュメントのリストを絞り込むために、ドキュメント取り込み時にメタデータを割り当て、フィルタリング・ファセット検索ができる
- Comprehendを利用して自動でメタデータを生成する

### 結論

- 自動で非構造データの絞り込み検索ができちゃう！

----

## アーキテクチャ

![](./architecture_kendra_comprehend.png "アーキテクチャ図")

----
![](./kendra_flow.png "利用の流れ")

----

## デモの流れ

（このファイルと同じディレクトリにある事前準備.ipynbを先に実行しておきます、デモでは割愛します）

1. Comprehendのデモ
2. Wikipediaから取得したファイルをディレクトリに配置する（図①事前準備で実施済み）
3. Kendraインデックスとデータソースコネクタをプロビジョニングします（事前準備で実施済み）
4. Comprehendを用いたエンティティ抽出（図②）
5. S3へのデータ配置（図③）
6. データソースのsync（図④）
7. 検索！（図⑤）

## 1. Comprehendのデモ

https://console.aws.amazon.com/comprehend/v2/home?region=us-east-1#home

## 2. Wikipediaから取得したファイルをディレクトリに配置する（図①事前準備で実施済み）

ファイルリストは以下。

In [1]:
!ls -1 Data/

Amazon.com.html
Amazon_Elastic_Compute_Cloud.html
Amazon_Simple_Queue_Service.html
Amazon_Simple_Storage_Service.html
Amazon_Web_Services.html
inference_pipeline_sparkml_xgboost_abalone.ipynb
Meta


## 3. Kendraインデックスとデータソースコネクタをプロビジョニングします（事前準備で実施済み）

新しい Kendraインデックス の作成には約15分ほどかかります。  
Cloudformationテンプレート の"./kendratemplate.yaml"で Kendraインデックス を作成しています。  

デモでは画面上でデータソースを作成する手順をご説明します。  

In [2]:
!sh provisioning.sh

comprehendlab-akazawt-20211118-14325
ComprehendDemo1117-002
{
    "StackId": "arn:aws:cloudformation:us-east-1:067150986393:stack/ComprehendDemo1117-002/f07e6b60-4816-11ec-bc26-12a634623527"
}


In [3]:
!aws cloudformation describe-stacks --stack-name "$(cat .stackname)" | grep StackStatus

            "StackStatus": "CREATE_COMPLETE",


In [10]:
!aws cloudformation describe-stacks --stack-name "$(cat .stackname)" --query "Stacks[0].Outputs[?OutputKey=='KendraDatasourceID'].OutputValue" --output text > .indexid

## 4. Comprehendを用いたエンティティ抽出（図②）

事前に配置してあるファイルを対象として、Comprehendでエンティティ抽出を行う。この結果を検索時のメタデータとして利用する。  

ファイルを逐次読み込み -> Comprehendのdetect_entitiesメソッドでエンティティを取得 -> 整形してjson化してファイルに格納

```python
# ファイル読み込み
    doc_text = open(fname, 'r', encoding="utf-8").read()
...
# detect_entitiesメソッドでエンティティを取得
    for i in range(0, len(doc_text), compre_text_size):
        try:
            entities = client.detect_entities(Text=doc_text[i:i+compre_text_size], LanguageCode='ja')
...
# json化
    metadata["_source_uri"] = "https://ja.wikipedia.org/wiki/" + npreplace
    attributes["Attributes"] = metadata
    print(json.dumps(attributes, sort_keys=True, indent=4))
```

In [4]:
# 1ファイルでメタデータ抽出を試してみる
#  元のWikipediaページ
#    https://ja.wikipedia.org/wiki/Amazon_Web_Services

!python3 getpagemetadata.py Data/Amazon_Web_Services.html

{
    "Attributes": {
        "COMMERCIAL_ITEM": [
            "Route",
            "EC2",
            "Prime Video",
            "Kindle",
            "Echo",
            "Kindle Fire",
            "Fire",
            "Fire HD 10",
            "Fire TV",
            "Prime Air"
        ],
        "DATE": [
            "2月",
            "3月",
            "4月",
            "5月",
            "6月",
            "7月",
            "8月",
            "2006年",
            "2004年",
            "2021年9月2日"
        ],
        "EVENT": [],
        "LOCATION": [
            "アメリカ東部",
            "北バージニア",
            "アメリカ西部",
            "カリフォルニア",
            "サンパウロ",
            "アイルランド",
            "ロンドン",
            "シンガポール",
            "大阪",
            "香港"
        ],
        "ORGANIZATION": [
            "Amazon Web Services",
            "Wikipedia",
            "Amazon_Web_Services",
            "Amazon.com",
            "アマゾン",
            "AWS",
            "Amazon",
            "日本政府

In [5]:
# 一括で全ファイルのメタデータ抽出を実行する
!sh ./getmetadata.sh

Generate metadata for 5 pages
Starting from 0
Working on 0
Working on Amazon_Web_Services
Working on Amazon.com
Working on Amazon_Simple_Queue_Service
Working on Amazon_Elastic_Compute_Cloud
Working on Amazon_Simple_Storage_Service
Successful completion at 0


## 5. S3へのデータ配置（図③）

In [6]:
# 事前配置してあるテキスト(html)データと作成したエンティティメタデータをS3にアップロードする
!sh upload.sh

upload: Data/Amazon_Web_Services.html to s3://comprehendlab-akazawt-20211118-14325/Data/Amazon_Web_Services.html
upload: Data/inference_pipeline_sparkml_xgboost_abalone.ipynb to s3://comprehendlab-akazawt-20211118-14325/Data/inference_pipeline_sparkml_xgboost_abalone.ipynb
upload: Data/Amazon_Simple_Queue_Service.html to s3://comprehendlab-akazawt-20211118-14325/Data/Amazon_Simple_Queue_Service.html
upload: Data/Amazon.com.html to s3://comprehendlab-akazawt-20211118-14325/Data/Amazon.com.html
upload: Data/Amazon_Elastic_Compute_Cloud.html to s3://comprehendlab-akazawt-20211118-14325/Data/Amazon_Elastic_Compute_Cloud.html
upload: Data/Amazon_Simple_Storage_Service.html to s3://comprehendlab-akazawt-20211118-14325/Data/Amazon_Simple_Storage_Service.html
upload: Meta/Data/Amazon_Simple_Storage_Service.txt.metadata.json to s3://comprehendlab-akazawt-20211118-14325/Meta/Data/Amazon_Simple_Storage_Service.txt.metadata.json
upload: Meta/Data/Amazon.com.txt.metadata.json to s3://comprehendlab-

## データソースのsync（図④）

現状、CloudFormationの[AWS::Kendra::DataSource](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/aws-properties-kendra-datasource-s3datasourceconfiguration.html#cfn-kendra-datasource-s3datasourceconfiguration-documentsmetadataconfiguration)でdefault languageに日本語を指定できないため、手動でデータソースの言語を日本語に設定する・・・

https://console.aws.amazon.com/kendra/home?region=us-east-1#indexes

In [7]:
!sh datasource_sync.sh

Kendra index ID is: fed3ec6e-c4c2-4c85-b1d6-ec5f81605dd5. Kendra data source id is: 159a19a5-54b1-4167-ae91-42b401734d14
{
    "ExecutionId": "00ef52b3-26d8-4de5-acc8-d9c86f50f3c1"
}
{
    "History": [
        {
            "ExecutionId": "00ef52b3-26d8-4de5-acc8-d9c86f50f3c1",
            "StartTime": 1637203022.127,
            "Status": "SYNCING",
            "Metrics": {
                "DocumentsAdded": "0",
                "DocumentsModified": "0",
                "DocumentsDeleted": "0",
                "DocumentsFailed": "0",
                "DocumentsScanned": "0"
            }
        }
    ]
}


## 検索！（図⑤）

現状、Kendraのコンソールからは[String List型のフィルタ（ファセット）検索](https://docs.aws.amazon.com/kendra/latest/dg/filtering.html)が利用できないため、SDKからクエリを実行する。

In [63]:
import boto3
import pprint

kendra = boto3.client('kendra')
#f = open('.indexid', 'r', encoding='UTF-8')
#index_id = f.read()

query='ジェフ'
index_id = "589a886f-aeec-40cd-9d9a-0b00415ce8a4"

query = kendra.query(
        QueryText = query,
        IndexId = index_id,
        AttributeFilter = {
                 "EqualsTo": { "Key": "_language_code", "Value": { "StringValue": "ja" }},
        }
)

print(query)

print ('\nSearch results for query: ' + query + '\n')        

for query_result in response['ResultItems']:

    print('-------------------')
    print('Type: ' + str(query_result['Type']))
        
    if query_result['Type']=='ANSWER' or query_result['Type'] == 'QUESTION_ANSWER':
        answer_text = query_result['DocumentExcerpt']['Text']
        print(answer_text)

    if query_result['Type']=='DOCUMENT':
        if 'DocumentTitle' in query_result:
            document_title = query_result['DocumentTitle']['Text']
            print('Title: ' + document_title)
        document_text = query_result['DocumentExcerpt']['Text']
        print(document_text)

    print ('------------------\n\n')  

{'QueryId': '26df836b-7066-422b-ad3f-0cdda3595dad', 'ResultItems': [{'Id': '26df836b-7066-422b-ad3f-0cdda3595dad-24d0437c-47fd-42c6-ab08-01b6f97ae99f', 'Type': 'DOCUMENT', 'AdditionalAttributes': [], 'DocumentId': 's3://comprehendlab-akazawt-20211117-22171/Data/Amazon.com.html', 'DocumentTitle': {'Text': 'Amazon.com - Wikipedia', 'Highlights': []}, 'DocumentExcerpt': {'Text': 'Amazon.comの論争\n\tZappos.com\n\tAmazon.co.jp\n\tジェフ・ベゾス\n\t通信販売\n\n\n関連文献[編集]\n\n\t横田増生『潜入ルポ\u3000アマゾン・ドット・コム(朝日文庫) 』朝日新聞出版、2010年\u3000ISBN 978-4022616845\n\tリチャード・ブラントー（訳：井口耕二）『ワンクリック―ジェフ・ベゾス率いるAmazonの隆盛』日経BPマーケティング、2012年\u3000ISBN 978-4822249151\n\tジャン＝バティスト・マレ『アマゾン、世界最良の企業潜入記』（En Amazonie. Infiltré dans le «meilleur des mondes»）', 'Highlights': [{'BeginOffset': 41, 'EndOffset': 44, 'TopAnswer': False, 'Type': 'STANDARD'}, {'BeginOffset': 158, 'EndOffset': 161, 'TopAnswer': False, 'Type': 'STANDARD'}]}, 'DocumentURI': 'https://s3.us-east-1.amazonaws.com/comprehendlab-akazawt-20211117-22171/Data/Amazon.com.html',

TypeError: must be str, not dict

In [ ]:

query = kendra.query(
        QueryText = query,
        IndexId = index_id,
        AttributeFilter = {'OrAllFilters':
                [
                         { "EqualsTo": { "Key": "_language_code", "Value": { "StringValue": "ja" }}},
                         { "EqualsTo": { "Key": "ORGANIZATION", "Value": { "StringListValue": [ "Amazon"] }}}
                ]
        }
)

In [ ]:
query = kendra.query(
        QueryText = query,
        IndexId = index_id,
        AttributeFilter = {'AndAllFilters': 
            [ 
                { "EqualsTo": { "Key": "_language_code", "Value": { "StringValue": "ja" }}},
            ]
            }
)